In [62]:
#notebook for TD HUU randomization report

In [63]:
# Imports
import os 
import pandas as pd
from django.core.exceptions import ValidationError
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from datetime import date
import csv
from django_pandas.io import read_frame

In [64]:
consents_pids = SubjectConsent.objects.all().values_list('screening_identifier', flat=True).distinct()
consents_pids = list(set(consents_pids))

# TD HUU Caregivers
td_huu_md = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected').values_list('study_maternal_identifier', flat=True)
td_huu_md = list(set(td_huu_md))


# TD Caregivers
td_md = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla')

#Consented HUU
consented_md = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected',
    screening_identifier__in=consents_pids)
consented_md_pids = consented_md.values_list('study_maternal_identifier', flat=True)
consented_md_pids = list(set(consented_md_pids))

#Unconsented HUU
td_huu_md = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected')
td_huu_md_pids = td_huu_md.values_list('study_maternal_identifier', flat=True)
td_huu_md_pids = list(set(td_huu_md_pids))
td_uncosent_pids = list(set(td_huu_md_pids)-set(consented_md_pids))

# TD HUU Offstudy
off_study_identifiers = CaregiverOffStudy.objects.values_list(
    'subject_identifier', flat=True)
consents = SubjectConsent.objects.filter(
    subject_identifier__in=off_study_identifiers).values_list(
    'screening_identifier', flat=True)
consents = list(set(consents))
offstudy = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected',
    screening_identifier__in=consents).values_list('study_maternal_identifier', flat=True)
offstudy = list(set(offstudy))

print("Total Consented Caregivers: ", len(consents_pids))
print("Total TD Caregivers: ", td_md.count())
print("Total TD HUU Caregivers: ", len(td_huu_md_pids))
print("Total TD Consented HUU: ", len(consented_md_pids))
print("Total TD Not Consented HUU: ", len(td_uncosent_pids))
print("Total TD Offstudy: ", len(offstudy))


data = [
    ['total_td_huu.csv', td_huu_md_pids],
    ['total_consented_td_huu.csv',  consented_md_pids],
    ['total_not_consented_td_huu.csv', td_uncosent_pids],
    ['total_offstudy_td_huu.csv', offstudy]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

Total Consented Caregivers:  1053
Total TD Caregivers:  388
Total TD HUU Caregivers:  114
Total TD Consented HUU:  47
Total TD Not Consented HUU:  67
Total TD Offstudy:  0


In [65]:
# TD Randomised HUU
td_md_identifier = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected').values_list('study_maternal_identifier', flat=True)
td_md_identifier = list(set(td_md_identifier))

randomised_worklist = WorkList.objects.filter(
    assigned__isnull=False,
    study_maternal_identifier__in=td_md_identifier).values_list(
        'study_maternal_identifier', flat=True)
randomised_worklist = list(set(randomised_worklist))

# Not Ranndomised
not_randomised_worklist = WorkList.objects.filter(
    assigned__isnull=True,
    study_maternal_identifier__in=td_md_identifier).values_list(
        'study_maternal_identifier', flat=True)
not_randomised_worklist = list(set(not_randomised_worklist))
final_not_randomised_worklist = []
for ident in not_randomised_worklist:
    log = LogEntry.objects.filter(study_maternal_identifier=ident)
    if not log:
        final_not_randomised_worklist.append(ident)
final_not_randomised_worklist = list(set(final_not_randomised_worklist))

# Missing worklist per prev study
worklist_identifiers = WorkList.objects.filter(
    study_maternal_identifier__in=td_md_identifier).values_list('study_maternal_identifier', flat=True)
worklist_identifiers = list(set(worklist_identifiers))
missing_worklist_identifiers = list(
    set(td_md_identifier) - set(worklist_identifiers))

# Worklist
worklist = WorkList.objects.filter(
    study_maternal_identifier__in=td_md_identifier).values_list(
        'study_maternal_identifier', flat=True)
worklist = list(set(worklist))

data_rando_dict = [
    worklist,randomised_worklist, final_not_randomised_worklist, missing_worklist_identifiers]

print("Total in the worklist: ", len(worklist))
print("Total randomised TD HUU: ", len(randomised_worklist))
print("Total Not Randomised TD HUU: ", len(final_not_randomised_worklist))
print("Total missing from worklist: ", len(missing_worklist_identifiers))

# importing library
import csv
 
    
data = [
    ['total_worklist_td_huu.csv', worklist],
    ['total_randomised_td_huu.csv', randomised_worklist],
    ['total_not_randomised_td_huu.csv', final_not_randomised_worklist],
    ['total_missing_worklist.csv', missing_worklist_identifiers]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

Total in the worklist:  114
Total randomised TD HUU:  72
Total Not Randomised TD HUU:  0
Total missing from worklist:  0


In [66]:
# Attempts
td_md_identifier = MaternalDataset.objects.filter(
    protocol='Tshilo Dikotla',
    mom_hivstatus='HIV-uninfected').values_list('study_maternal_identifier', flat=True)
td_md_identifier = list(set(td_md_identifier))

qs = LogEntry.objects.filter(study_maternal_identifier__in=td_md_identifier)
attempt_identifiers = qs.values_list(
        'study_maternal_identifier', flat=True)
attempt_identifiers = list(set(attempt_identifiers))

# Not attempted
not_attempted = list(set(td_md_identifier) - set(attempt_identifiers))

data_attemps_dict = {
    'TD HUU attempts': attempt_identifiers,
    'TD HUU Not attempted': not_attempted
}

print("Attempts made: ", len(attempt_identifiers))
print('No Attempts: ', len(not_attempted))


data = [
    ['total_attempts_td_huu.csv', attempt_identifiers],
    ['total_notattempted_td_huu.csv',  not_attempted],
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

Attempts made:  113
No Attempts:  1


In [67]:
# Continued Contact
continued_contact = []
for identifier in attempt_identifiers:
    obj = LogEntry.objects.filter(
        study_maternal_identifier=identifier).order_by('-created')
    if obj:
        if obj[0].appt in ['thinking']:
            continued_contact.append(identifier)
        elif obj[0].may_call == 'No' and not obj[0].appt in ['No']:
            continued_contact.append(identifier)
    logs = LogEntry.objects.filter(
        phone_num_success=['none_of_the_above'],
        study_maternal_identifier=identifier).order_by('-created')
    if logs:
        if logs.count() <= 3:
            if logs[0].final_contact == 'No':
                continued_contact.append(identifier)
continued_contact = list(set(continued_contact) - set(consented_md_pids))

# All not continued contract and not consented

not_continued_contact_not_consented = list(set(td_uncosent_pids)-set(continued_contact))

qs = LogEntry.objects.filter(study_maternal_identifier__in=not_continued_contact_not_consented)

qs_data = []

for obj in qs:
    obj_dict = obj.__dict__
    del obj_dict['_state']
    del obj_dict['device_created']
    del obj_dict['device_modified']
    del obj_dict['modified']
    del obj_dict['created']
    del obj_dict['revision']    
    del obj_dict['user_created']
    del obj_dict['user_modified']
    del obj_dict['hostname_created']
    del obj_dict['hostname_modified']
    del obj_dict['log_id']
    
    qs_data.append(obj_dict)

df = pd.DataFrame(qs_data)
df.to_csv('log_entries_no_consent_not_continued_contact.csv', sep=',', encoding='utf-8')


data_attemps_dict = {'TD HUU Continued Contact': len(continued_contact)}

print('TD HUU Continued Contact: ', len(continued_contact))
print('TD HUU Not COntinued Contact Not Consented: ', len(not_continued_contact_not_consented))

data = [
    ['total_continued_contact_td_huu.csv', continued_contact],
    ['total_no_concent_not_continued_contact_td_huu.csv', not_continued_contact_not_consented]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

TD HUU Continued Contact:  21
TD HUU Not COntinued Contact Not Consented:  46


In [68]:
unreacheble = []
for identifier in attempt_identifiers:
    logs = LogEntry.objects.filter(
        phone_num_success=['none_of_the_above'],
        study_maternal_identifier=identifier).order_by('-created')
    if logs.count() >= 3:
        if (logs[0].phone_num_success == ['none_of_the_above'] and
            logs[1].phone_num_success == ['none_of_the_above'] and 
            logs[2].phone_num_success == ['none_of_the_above'] and
            logs[0].final_contact == 'Yes'):
            unreacheble.append(identifier)
unreacheble = list(set(unreacheble) - set(consented_md_pids))


print('TD HUU Unreacheble: ', len(unreacheble))

data = [
    ['total_unreacheble_td_huu.csv', unreacheble]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

TD HUU Unreacheble:  0


In [ ]:
# Declined from call log
declined_do_not_call = LogEntry.objects.filter(
    study_maternal_identifier__in=attempt_identifiers,
    may_call__iexact='No').values_list(
        'study_maternal_identifier', flat=True)
declined_do_not_call = list(set(declined_do_not_call))

# Screening rejects
screened_identifiers = ScreeningPriorBhpParticipants.objects.filter(
    flourish_participation='No',
    study_maternal_identifier__in=attempt_identifiers).values_list(
        'study_maternal_identifier', flat=True)
screened_identifiers = list(set(screened_identifiers))

# Merge the 2 above lists
identifiers = screened_identifiers + declined_do_not_call
identifiers = list(set(identifiers) - set(consented_md_pids))

print('TD HUU Declined: ', len(identifiers))

data = [
    ['total_declined_td_huu.csv', identifiers]
]

for d in data:
    with open(d[0], 'w') as f:
        writer = csv.writer(f)
        for val in d[1]:
            writer.writerow([val])

TD HUU Declined:  13
